In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../../../../')
print(os.getcwd())
from v2.strategy.indicators.optimal_v2 import Optimal_v2
from v2.model import Trading
from v2.strategy.indicators.param import Param
from v2.strategy.indicators.roc import RateOfChange
from v2.research.scripts.notebook_utils import notebookUtils
from v2.research.scripts.scoreboard import updateScoreboard
from sklearn.model_selection import train_test_split

from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import confusion_matrix

/Users/rosscopeland/Desktop/personal/code/vivaldi/back_testing


In [2]:
nu = notebookUtils()
model_name = "lof_test"

In [3]:
dataset, features, indicators = nu.loadData(indicators=[('moh', 'time', ''), ('tod', 'time', ''), ('dow', 'time', ''), ('bop', 'close', ''),('optimal_v2', 'close', '')],
                            param_spec={},
                            optimal_threshold={'buy':(0.005, 0.03)},
                            spans=[{'indicator_name':'willr',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [10,30]},
                                    {'indicator_name':'cci',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [10,30,60]},
                                    {'indicator_name':'rsi',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [2,3,5,10,30]},
                                    {'indicator_name':'natr',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [30,60,120,180]},
                                    {'indicator_name':'cmo',
                                    'column_name': 'close',
                                    'param_name': 'period',
                                    'param_values': [2,3,5,10,60]}
                                    ],
                            scale='minmaxwindow')
        
    # create one minute, five minute, ten minute, and twenty minute slopes for macd, rsi

Loading data from OMG...
Loading data from chunk 0...
Loading data from ONE...
Loading data from chunk 0...
Loading data from ADA...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from ETH...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from chunk 4...
Loading data from chunk 5...
Loading data from NANO...
Loading data from chunk 0...
Loading data from EOS...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from VET...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from ICX...
Loading data from chunk 0...
Loading data from chunk 1...
Loading data from chunk 2...
Loading data from chunk 3...
Loading data from XRP...
Loading data from ch

In [4]:
dataset.drop(["time", "open", "high", "low", "volume"], axis=1, inplace=True)
dataset.dropna(inplace=True)
dataset

,close,MOH,TOD,DOW,BOP,optimal,WILLR_close_period_10,WILLR_close_period_30,CCI_close_period_10,CCI_close_period_30,...,RSI_close_period_30,NATR_close_period_30,NATR_close_period_60,NATR_close_period_120,NATR_close_period_180,CMO_close_period_2,CMO_close_period_3,CMO_close_period_5,CMO_close_period_10,CMO_close_period_60
15179,2.065706,0,0,0,0.500000,0.0,0.111967,0.000559,3.643429e-01,0.466158,...,0.406191,0.158824,0.202346,0.234607,0.272644,1.062345e-04,0.019543,0.156660,0.291282,0.507014
15180,2.065703,0,0,0,0.500000,0.0,0.111967,0.000280,3.888805e-01,0.467864,...,0.406114,0.152851,0.197851,0.231280,0.269771,8.592732e-05,0.018208,0.155202,0.290980,0.506961
15181,2.065702,0,0,0,0.500000,0.0,0.097669,0.000140,4.058662e-01,0.469289,...,0.406074,0.147064,0.193422,0.227975,0.266909,7.213788e-05,0.017321,0.154304,0.290813,0.506934
15182,2.065201,0,0,0,1.000000,0.0,0.524853,0.047308,6.399915e-12,0.460001,...,0.393728,0.145858,0.192039,0.226709,0.265759,7.204705e-07,0.000735,0.047703,0.242683,0.498549
15183,2.064950,0,0,0,0.500000,0.0,0.340852,0.027591,2.414962e-01,0.460373,...,0.387577,0.142294,0.189056,0.224358,0.263692,3.620432e-07,0.000428,0.033306,0.222117,0.494355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11372817,9.530607,1,0,0,0.675000,0.0,0.337968,0.531603,3.089283e-01,0.512159,...,0.422342,0.203818,0.360426,0.531232,0.645109,7.832983e-02,0.129377,0.249729,0.393035,0.460296
11372818,9.483304,0,0,0,0.000000,0.0,0.355571,0.355351,1.736127e-01,0.338302,...,0.301771,0.249090,0.398570,0.561424,0.671768,6.413275e-03,0.023645,0.081091,0.196951,0.384190
11372819,9.478652,0,0,0,0.816667,0.0,0.416996,0.416834,2.329341e-01,0.293968,...,0.292522,0.277890,0.422496,0.579520,0.687048,5.432314e-03,0.021100,0.074853,0.186561,0.377557
11372820,9.477326,0,0,0,0.814815,0.0,0.408444,0.408274,3.184019e-01,0.337347,...,0.289875,0.298662,0.440049,0.592852,0.698290,4.996627e-03,0.020172,0.072855,0.183487,0.375663


In [5]:
X = dataset.iloc[:int(len(dataset)*0.8)]
y = dataset.iloc[-int(len(dataset)*0.2):]
y

,close,MOH,TOD,DOW,BOP,optimal,WILLR_close_period_10,WILLR_close_period_30,CCI_close_period_10,CCI_close_period_30,...,RSI_close_period_30,NATR_close_period_30,NATR_close_period_60,NATR_close_period_120,NATR_close_period_180,CMO_close_period_2,CMO_close_period_3,CMO_close_period_5,CMO_close_period_10,CMO_close_period_60
9107365,9.649866,0,0,0,0.552147,0.0,0.948393,0.626406,0.592020,0.530581,...,0.462672,0.071668,0.056957,0.035715,0.025356,0.999105,0.981213,0.863554,0.622918,0.455412
9107366,9.624933,0,0,0,0.000000,0.0,0.476506,0.306245,0.212384,0.404273,...,0.360293,0.085015,0.065550,0.041617,0.030201,0.069067,0.133627,0.229863,0.327127,0.386983
9107367,9.634717,0,0,0,0.679054,0.0,0.669848,0.480437,0.344819,0.442300,...,0.409455,0.095493,0.072283,0.046061,0.033637,0.462054,0.425641,0.433642,0.442249,0.418266
9107368,9.630608,0,0,0,0.500000,0.0,0.588661,0.421937,0.382140,0.453547,...,0.394698,0.093300,0.071487,0.045781,0.033450,0.341117,0.351094,0.380677,0.409246,0.407898
9107369,9.634704,0,0,0,0.994872,0.0,0.669603,0.480261,0.430239,0.475669,...,0.414449,0.094836,0.072877,0.046841,0.034259,0.567065,0.485769,0.462402,0.454047,0.420633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11372817,9.530607,1,0,0,0.675000,0.0,0.337968,0.531603,0.308928,0.512159,...,0.422342,0.203818,0.360426,0.531232,0.645109,0.078330,0.129377,0.249729,0.393035,0.460296
11372818,9.483304,0,0,0,0.000000,0.0,0.355571,0.355351,0.173613,0.338302,...,0.301771,0.249090,0.398570,0.561424,0.671768,0.006413,0.023645,0.081091,0.196951,0.384190
11372819,9.478652,0,0,0,0.816667,0.0,0.416996,0.416834,0.232934,0.293968,...,0.292522,0.277890,0.422496,0.579520,0.687048,0.005432,0.021100,0.074853,0.186561,0.377557
11372820,9.477326,0,0,0,0.814815,0.0,0.408444,0.408274,0.318402,0.337347,...,0.289875,0.298662,0.440049,0.592852,0.698290,0.004997,0.020172,0.072855,0.183487,0.375663


In [6]:
contamination_val = (len(X[X['optimal'] > 0.005]) / len(X))*3/2
contamination_val

0.019360667792705177

In [9]:
clf = LocalOutlierFactor(contamination=contamination_val, n_jobs=-1).fit(X)

In [ ]:
preds = clf.predict(y)